### 结构化生成回答

#### 1. 介绍

**JSON**（JavaScript Object Notation）是当前使用最广泛的数据交换格式之一。**结构化输出**是一个功能，确保模型生成的响应符合你提供的**JSON Schema**，从而避免遗漏必要的键或生成无效的枚举值。

**优点**：
- **可靠的类型安全**：无需担心格式错误的响应。
- **明确的拒绝**：可以程序化检测模型的拒绝响应。
- **更简单的提示**：不需要复杂的提示来实现一致的格式化。

我们既可以在prompt提示中设置所需的结构化模型的输出，OPENAI也单独为模型提供了Json架构与可选参数。（3种）

这也展现了大模型应用开发的发展历程和应用趋势。

**教程中为方便展示已把description描述和prompt设置成中文方便观看，在真实的业务场景中想要优化效果需要改成英文的**

#### 2. 获取结构化响应

##### 2.1 最早期 用prompt控制生成特定的格式（非常不推荐）

prompt与fewshot生成QA问答对：

例如要求模型生成特定数量的问答对，并可以分隔问题和答案（课程辅导等，书籍提取）：

In [17]:
import json
import openai

# 1. 初始化 OpenAI 客户端
client = openai.OpenAI()

# 2. 准备提示信息和格式说明
messages = [
    {
        "role": "system", 
        "content": "你是一个助手。请以JSON格式返回3个关于大模型原理Decoder的问答对。必须使用以下格式：[{\"question\": \"问题1\", \"answer\": \"答案1\"}, {\"question\": \"问题2\", \"answer\": \"答案2\"}, {\"question\": \"问题3\", \"answer\": \"答案3\"}]"
    },
    {
        "role": "user", 
        "content": "生成3个关于大模型原理Decoder的问答对"
    }
]

try:
    # 3. 发送请求
    response = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=messages,
        # response_format=
    )
    
    # 打印原始响应内容（调试用）
    print("Raw response:", response.choices[0].message.content)
    
    # 4. 解析响应
    try:
        qa_pairs = json.loads(response.choices[0].message.content)
        
        # 5. 打印结果
        for pair in qa_pairs:
            print(f"Q: {pair['question']}")
            print(f"A: {pair['answer']}\n")
            
    except json.JSONDecodeError as e:
        print(f"JSON解析错误: {e}")
        print(f"收到的内容: {response.choices[0].message.content}")
        
except Exception as e:
    print(f"API请求错误: {e}")

Raw response: [
    {"question": "Decoder在大模型中有什么作用？", "answer": "Decoder负责将输入的隐藏状态转换为输出序列，它逐步生成每个词，并根据之前生成的词和上下文信息预测下一个词。"},
    {"question": "Decoder是如何处理输入序列的？", "answer": "Decoder接收来自编码器的上下文向量，并结合自身的输入和内部状态，通过自注意力机制和前馈神经网络生成最终的输出序列。"},
    {"question": "在训练Decoder时常用的损失函数是什么？", "answer": "在训练Decoder时，通常使用交叉熵损失函数来衡量生成序列与目标序列之间的差异，通过最小化此损失函数来优化模型参数。"}
]
Q: Decoder在大模型中有什么作用？
A: Decoder负责将输入的隐藏状态转换为输出序列，它逐步生成每个词，并根据之前生成的词和上下文信息预测下一个词。

Q: Decoder是如何处理输入序列的？
A: Decoder接收来自编码器的上下文向量，并结合自身的输入和内部状态，通过自注意力机制和前馈神经网络生成最终的输出序列。

Q: 在训练Decoder时常用的损失函数是什么？
A: 在训练Decoder时，通常使用交叉熵损失函数来衡量生成序列与目标序列之间的差异，通过最小化此损失函数来优化模型参数。



可能会正好的json，也可能会因为多了多余的字符无法直接用代码解析：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241031163440.png)

号外——官方出手了：




### 2.2 使用 response_format 参数生成结构化输出（非常推荐）


有两种定义数据结构的方式，一种是用类定义（Pydantic），一种是用使用 JSON Schema 字典（tool use时接触的）

## 方式一：使用 Pydantic BaseModel

Pydantic 是一个数据验证库，通过继承 BaseModel 来定义数据结构：

```python
from pydantic import BaseModel
from typing import List

class QAPair(BaseModel):
    question: str
    answer: str

# 使用示例
response_format = QAPair
```

优点：
- 提供完整的数据验证功能
- 自动类型转换，支持嵌套结构
- 与 OpenAI API 完全兼容，官方给出的最佳实践

## 方式二：直接使用 JSON Schema 字典

这种方式直接定义 JSON Schema：

```python
schema = {
    "type": "object",
    "properties": {
        "question": {"type": "string"},
        "answer": {"type": "string"}
    },
    "required": ["question", "answer"],
    "additionalProperties": false
}

# 使用示例
response_format = {"type": "json_schema", "schema": schema}
```

优点：
- 直接使用 JSON Schema 规范
- 灵活的结构定义，直白的添加更多验证规则


#### 2.2.1 用JSON Schema生成结构化输出

**步骤 1：定义架构**

设计模型应遵循的 JSON Schema。确保清晰命名键，并为重要字段添加描述。

**步骤 2：提供架构**

在 API 调用中指定架构：

In [12]:
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

**优化：以类型安全的方式使用生成的数据**（旧方法）

确保解析生成的数据为对应类型，使用 Pydantic 进行类型验证。

In [16]:
from pydantic import BaseModel, ValidationError
from typing import List

class Step(BaseModel):
    explanation: str
    output: str

class Solution(BaseModel):
    steps: List[Step]
    final_answer: str

try:
    solution = Solution.parse_raw(response.choices[0].message.content)
    print(solution)
except ValidationError as e:
    print(e.json())

steps=[Step(explanation='We start with the equation: \\(8x + 7 = -23\\). Our goal is to isolate \\(x\\), so the first step is to eliminate the constant term on the left side.', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to begin isolating the variable term \\(8x\\).', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides. On the left, the \\(+7\\) and \\(-7\\) cancel each other out, leaving \\(8x\\).', output='8x = -30'), Step(explanation='Now, divide both sides by 8 to solve for \\(x\\).', output='\\frac{8x}{8} = \\frac{-30}{8}'), Step(explanation='Simplify \\(\\frac{-30}{8}\\) by dividing the numerator and the denominator by their greatest common divisor, which is 2.', output='x = \\frac{-30 \\div 2}{8 \\div 2} = \\frac{-15}{4}')] final_answer='x = \\frac{-15}{4}'


输出示例：
```json
{
  "steps": [
    {
      "explanation": "Start with the equation 8x + 7 = -23.",
      "output": "8x + 7 = -23"
    },
    {
      "explanation": "Subtract 7 from both sides to isolate the term with the variable.",
      "output": "8x = -23 - 7"
    }
  ],
  "final_answer": "x = -15 / 4"
}
```

号外——官方出手了：加入strict了参数限制

In [ ]:
response_format: { type: "json_schema", json_schema: {"strict": true, "schema": ...} }

优化： 拒绝情况

如果模型拒绝响应，将包含一个新字段 `refusal`，用于指示拒绝的原因。可在 UI 中处理此情况。

In [ ]:
math_reasoning = response.choices[0].message
if (math_reasoning.refusal):
    print(math_reasoning.refusal)
else:
    print(math_reasoning.parsed)

 提示与最佳实践

**Schema 限制**:
- 最多100个对象属性
- 最多5层嵌套
- 字符串长度总和不超过15,000字符
- 枚举值最多500个

- 处理用户输入时，包含有关如何处理无效响应的说明。
- 对于错误，调整提示并尝试将任务拆分为更简单的子任务。


支持的模式

结构化输出支持 JSON Schema 的子集包括字符串、数字、布尔值、对象、数组等类型。所有字段必须为必填项，并且不允许存在额外属性。

#### 2.2.2 用 Pydantic BaseModel来定义生成结构化输出


Pydantic 提供了一种简单且类型安全的方法来验证和解析数据。在后端写接口时有用到它来校验前后端传输的数据格式是否正确，现在OPENAI将它也纳入了内嵌的使用当中。

用一个具体例子来说明，创建一个日历事件的模型：

- **步骤**：
  1. 定义一个数据模型（`CalendarEvent`）。
  2. 调用 OpenAI API，并指定 `response_format` 为 `CalendarEvent`。
  3. 解析响应中的信息。

In [20]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "秋天了，我想在周末去爬山看枫叶"},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed
event

CalendarEvent(name='山岳秋季观赏', date='10月的一个周末', participants=['自己'])